In [34]:
import os 
from dotenv import load_dotenv
from openai import OpenAI
import json 

api_key = os.environ.get('OPENAI_API_KEY')
# Open ai API를 사용하기 위한 클라이언트 객체생성
client = OpenAI(api_key=api_key)


load_dotenv()

# 헬퍼 함수
def show_json(obj):
    #obj의 모델을 Json 형태로 변환 후 출력
    display(json.loads(obj.model_dump_json()))


In [35]:
assistant_list = client.beta.assistants.list()

for assistant in assistant_list:
    print(f"Assistant Name: {assistant.name}, Assistant ID: {assistant.id}")


Assistant Name: 상표 식별력 판단 AI(GPT-4o), Assistant ID: asst_mD9MAguey0mzXs0wKEJmG4lV
Assistant Name: 상표 식별력 판단 AI, Assistant ID: asst_GDlNLfM4j2LCpTYFgULSR1V6


In [47]:
# assistant id를 별도의 변수에 담음
ASSISTANT_ID = 'asst_n01Ro5AKLNdP8Ye4IZqB727G'

print(f"[새로 생성한 ASSISTANT_ID]\n{ASSISTANT_ID}")


[새로 생성한 ASSISTANT_ID]
asst_n01Ro5AKLNdP8Ye4IZqB727G


# assistant update & functions schema 

### function과 schema

In [61]:
def return_ai_company():
    return '(주)에이아이노미스'

function_schema={
    'name' : 'return_ai_nomis',
    'description' : "모든 응답 마지막에 반환한 문자열 문구를 추가합니다",
    'parameters':{
        'type': 'object',
        'properties': {},
        'additionalProperties':False
    },
    'strict': True # 모든 응답에서 호출되도록
}

### 새로운 어시스턴트 생성

In [46]:
assistant = client.beta.assistants.update(
    assistant_id=ASSISTANT_ID,
    instructions='당신은 수학의 천재 입니다. 사용자의 질문에 계산하여 대답하세요.',
    tools= [
        {'type': 'code_interpreter'},
        {'type': "file_search"},
        {'type': 'function', 'function': function_schema}
    ],
    model='gpt-4o-mini'
)
show_json(assistant)

{'id': 'asst_n01Ro5AKLNdP8Ye4IZqB727G',
 'created_at': 1725613108,
 'description': None,
 'instructions': '당신은 수학의 천재 입니다. 사용자의 질문에 계산하여 대답하세요.',
 'metadata': {},
 'model': 'gpt-4o-mini',
 'name': '수학천재',
 'object': 'assistant',
 'tools': [{'type': 'code_interpreter'},
  {'type': 'file_search',
   'file_search': {'max_num_results': None,
    'ranking_options': {'ranker': 'default_2024_08_21',
     'score_threshold': 0.0}}},
  {'function': {'name': 'return_ai_nomis',
    'description': "모든 응답 마지막에 '(주)에이아이노미스' 문구를 추가합니다",
    'parameters': {'type': 'object',
     'properties': {},
     'additionalProperties': False},
    'strict': True},
   'type': 'function'}],
 'response_format': 'auto',
 'temperature': 1.0,
 'tool_resources': {'code_interpreter': {'file_ids': []},
  'file_search': {'vector_store_ids': []}},
 'top_p': 1.0}

In [49]:
client.beta.vector_stores.list()

SyncCursorPage[VectorStore](data=[VectorStore(id='vs_lnyjqbRPhkqR5RkQ3Y3pdiN1', created_at=1725525376, file_counts=FileCounts(cancelled=0, completed=0, failed=0, in_progress=0, total=0), last_active_at=1725610130, metadata={}, name='상표 식별 documents', object='vector_store', status='completed', usage_bytes=0, expires_after=None, expires_at=None), VectorStore(id='vs_8HRlMlOmpLEGKXIJAPZBGn2w', created_at=1725507210, file_counts=FileCounts(cancelled=0, completed=4, failed=0, in_progress=0, total=4), last_active_at=1725612858, metadata={}, name='상표 식별 documents', object='vector_store', status='completed', usage_bytes=8461165, expires_after=None, expires_at=None)], object='list', first_id='vs_lnyjqbRPhkqR5RkQ3Y3pdiN1', last_id='vs_8HRlMlOmpLEGKXIJAPZBGn2w', has_more=False)

In [50]:
thread = client.beta.threads.create()

show_json(thread)

{'id': 'thread_WVK6e494ZdBI52zNrhDHOI8y',
 'created_at': 1725613146,
 'metadata': {},
 'object': 'thread',
 'tool_resources': {'code_interpreter': None, 'file_search': None}}

In [51]:
message = client.beta.threads.messages.create(
    thread_id= thread.id,
    role='user',
    content='“x^2 + y^2가 최소가 되도록 x + y = 10일 때 x와 y의 값을 찾아줘.”'
)
show_json(message)

{'id': 'msg_v7wDoVZls3mEWGP93I0YH2NK',
 'assistant_id': None,
 'attachments': [],
 'completed_at': None,
 'content': [{'text': {'annotations': [],
    'value': '“x^2 + y^2가 최소가 되도록 x + y = 10일 때 x와 y의 값을 찾아줘.”'},
   'type': 'text'}],
 'created_at': 1725613217,
 'incomplete_at': None,
 'incomplete_details': None,
 'metadata': {},
 'object': 'thread.message',
 'role': 'user',
 'run_id': None,
 'status': None,
 'thread_id': 'thread_WVK6e494ZdBI52zNrhDHOI8y'}

In [52]:
run = client.beta.threads.runs.create(
    thread_id=thread.id, # 생성한스레드(카톡방)
    assistant_id=ASSISTANT_ID # 적용할 AssistantID
)
show_json(run)

{'id': 'run_7BnkU03e8qItCGDdPatXUUVu',
 'assistant_id': 'asst_n01Ro5AKLNdP8Ye4IZqB727G',
 'cancelled_at': None,
 'completed_at': None,
 'created_at': 1725613223,
 'expires_at': 1725613823,
 'failed_at': None,
 'incomplete_details': None,
 'instructions': '당신은 수학의 천재 입니다. 사용자의 질문에 계산하여 대답하세요.',
 'last_error': None,
 'max_completion_tokens': None,
 'max_prompt_tokens': None,
 'metadata': {},
 'model': 'gpt-4o-mini',
 'object': 'thread.run',
 'parallel_tool_calls': True,
 'required_action': None,
 'response_format': 'auto',
 'started_at': None,
 'status': 'queued',
 'thread_id': 'thread_WVK6e494ZdBI52zNrhDHOI8y',
 'tool_choice': 'auto',
 'tools': [{'type': 'code_interpreter'},
  {'type': 'file_search',
   'file_search': {'max_num_results': None,
    'ranking_options': {'ranker': 'default_2024_08_21',
     'score_threshold': 0.0}}},
  {'function': {'name': 'return_ai_nomis',
    'description': "모든 응답 마지막에 '(주)에이아이노미스' 문구를 추가합니다",
    'parameters': {'type': 'object',
     'properties': {},


In [53]:
import time 

def wait_on_run(run, thread):
    # 주어진 실행 (run)이 완료 될때까지 대기
    # status 가 'queued' 또는 'inprogress'인 경우에는 계속 polling 하며 대기
    while run.status == 'queued' or run.status == 'in_progress':
        # run.status를 업데이트 합니다.
        run = client.beta.threads.runs.retrieve(
            thread_id = thread.id,
            run_id= run.id
        )
        time.sleep(0.5)
    return run

In [54]:

# run 객체를 대기 상태로 설정하고, 해당 스레드에서 실행을 완료할 때가지 기다림
run = wait_on_run(run, thread)

# status가 'complete'인 경우에는 결과를 출력합니다.
show_json(run)

{'id': 'run_7BnkU03e8qItCGDdPatXUUVu',
 'assistant_id': 'asst_n01Ro5AKLNdP8Ye4IZqB727G',
 'cancelled_at': None,
 'completed_at': 1725613230,
 'created_at': 1725613223,
 'expires_at': None,
 'failed_at': None,
 'incomplete_details': None,
 'instructions': '당신은 수학의 천재 입니다. 사용자의 질문에 계산하여 대답하세요.',
 'last_error': None,
 'max_completion_tokens': None,
 'max_prompt_tokens': None,
 'metadata': {},
 'model': 'gpt-4o-mini',
 'object': 'thread.run',
 'parallel_tool_calls': True,
 'required_action': None,
 'response_format': 'auto',
 'started_at': 1725613223,
 'status': 'completed',
 'thread_id': 'thread_WVK6e494ZdBI52zNrhDHOI8y',
 'tool_choice': 'auto',
 'tools': [{'type': 'code_interpreter'},
  {'type': 'file_search',
   'file_search': {'max_num_results': None,
    'ranking_options': {'ranker': 'default_2024_08_21',
     'score_threshold': 0.0}}},
  {'function': {'name': 'return_ai_nomis',
    'description': "모든 응답 마지막에 '(주)에이아이노미스' 문구를 추가합니다",
    'parameters': {'type': 'object',
     'properti

In [55]:
run_steps = client.beta.threads.runs.steps.list(
    thread_id=thread.id, run_id = run.id, order='asc'
)

In [56]:
for step in run_steps.data:
    #각 세부 단계의 정보를 불러옴
    step_detail = step.step_details
    #세부 정보를 json형식으로 출력
    show_json(step_detail)

{'tool_calls': [{'id': 'call_Wphxw2vSIC2d5FJFV0JfR15G',
   'code_interpreter': {'input': "import sympy as sp\n\n# 변수 정의\nx, y = sp.symbols('x y')\n\n# 제약 조건\nconstraint = x + y - 10\n\n# 목표 함수\nobjective_function = x**2 + y**2\n\n# 제약 조건을 대체하여 목적 함수를 단일 변수로 변환\ny_expr = 10 - x\nsubstituted_objective = objective_function.subs(y, y_expr)\n\n# 목적 함수를 미분하여 최적화\nderivative = sp.diff(substituted_objective, x)\ncritical_points = sp.solve(derivative, x)\n\n# x 값을 이용하여 y 값을 찾기\nx_value = critical_points[0]\ny_value = 10 - x_value\n\nx_value, y_value",
    'outputs': []},
   'type': 'code_interpreter'}],
 'type': 'tool_calls'}

{'message_creation': {'message_id': 'msg_rtiOmpGGguOLkDVuRSNJLNhb'},
 'type': 'message_creation'}

In [57]:
# thread.id 를 사용하여 메시지 목록을 가져옴
messages = client.beta.threads.messages.list(thread_id=thread.id)

#결과 출력
show_json(messages)

{'data': [{'id': 'msg_rtiOmpGGguOLkDVuRSNJLNhb',
   'assistant_id': 'asst_n01Ro5AKLNdP8Ye4IZqB727G',
   'attachments': [],
   'completed_at': None,
   'content': [{'text': {'annotations': [],
      'value': '\\( x + y = 10 \\)일 때 \\( x^2 + y^2 \\)가 최소가 되도록 하는 \\( x \\)와 \\( y \\)의 값은 각각 \\( 5 \\)입니다. 즉, \\( x = 5 \\) 및 \\( y = 5 \\)입니다. (주)에이아이노미스'},
     'type': 'text'}],
   'created_at': 1725613229,
   'incomplete_at': None,
   'incomplete_details': None,
   'metadata': {},
   'object': 'thread.message',
   'role': 'assistant',
   'run_id': 'run_7BnkU03e8qItCGDdPatXUUVu',
   'status': None,
   'thread_id': 'thread_WVK6e494ZdBI52zNrhDHOI8y'},
  {'id': 'msg_v7wDoVZls3mEWGP93I0YH2NK',
   'assistant_id': None,
   'attachments': [],
   'completed_at': None,
   'content': [{'text': {'annotations': [],
      'value': '“x^2 + y^2가 최소가 되도록 x + y = 10일 때 x와 y의 값을 찾아줘.”'},
     'type': 'text'}],
   'created_at': 1725613217,
   'incomplete_at': None,
   'incomplete_details': None,
   'metadata':